In [ ]:
"""
Enhanced NUA Detector Pipeline - ZMODYFIKOWANY I POPRAWIONY
Cel: 1) POMINIĘCIE generowania i dzielenia datasetu.
     2) Bezpośrednie wczytanie istniejącego splitu z: /content/drive/MyDrive/NUA_Detection_Results/split
     3) Wytrenowanie detektora CNN (EfficientNetV2-M) + ekstrakcja cech + SVM + meta-ensemble.
     4) Wygenerowanie metryk i raportu.

Poprawka błędu: Usunięto argument 'verbose=True' z ReduceLROnPlateau w celu zapewnienia kompatybilności z nowszymi wersjami PyTorch.

Instrukcje: Uruchomić w środowisku z dostępem do dysku Google Colab z pakietami:
 pip install timm torchvision scikit-learn matplotlib tqdm pillow exifread scikit-image
"""

import os
import shutil
import random
import time
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image, ImageStat, ImageEnhance, ImageFilter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import timm

from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, roc_curve, auc, classification_report)
import matplotlib.pyplot as plt
from skimage import exposure, filters # Dodane do symulacji NUA
import exifread
import joblib
from torchvision.datasets import ImageFolder # Przeniesione na górę dla porządku

# ==============
# Ustawienia
# ==============
# Zaktualizowana ścieżka do czystych zdjęć (Cover)
SOURCE_DIR = Path("/content/drive/MyDrive/stegano_dataset/cover")
OUTPUT_ROOT = Path("/content/drive/MyDrive/NUA_Detection_Results")
DATASET_DIR = OUTPUT_ROOT / "dataset"  # dataset/original and dataset/nua
GENERATED_NUA_DIR = OUTPUT_ROOT / "generated_nua"
PROPOSED_LABELS_CSV = OUTPUT_ROOT / "proposed_labels.csv"
IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.tif', '.tiff', '.webp'}
SEED = 42
BATCH_SIZE = 12
IMG_SIZE = (320, 320)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2  # 0 = original, 1 = NUA
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == 'cuda':
    torch.cuda.manual_seed_all(SEED)

# Poprawki: Zwiększenie epok i mniejszy Learning Rate dla fine-tune
NUM_EPOCHS = 15
LEARNING_RATE = 3e-5

# =================================================================
# Krok 1A: Generowanie obrazów NUA (FUNKCJA NIEUŻYWANA W NOWYM PIPELINE)
# =================================================================

def apply_nua_effect(img: Image, method: str):
    """Aplikuje symulowany efekt NUA do obrazu PIL."""
    try:
        if method == 'simulated_hdr':
            # Symulacja HDR: zwiększenie kontrastu i wyostrzenie
            enhancer = ImageEnhance.Contrast(img)
            img = enhancer.enhance(1.8)
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(1.5)

        elif method == 'ai_denoising_sharp':
            # Symulacja AI denoising i wyostrzania
            np_img = np.array(img).astype(np.uint8)
            # Lekki szum Gaussa (symulacja sygnału wejściowego)
            noise = np.random.normal(0, 10, np_img.shape).astype(np.int16)
            np_img = np.clip(np_img + noise, 0, 255).astype(np.uint8)

            # Proste odszumianie (simulacja) - używamy Median Filter
            img = Image.fromarray(np_img).filter(ImageFilter.MedianFilter(size=3))

            # Agresywne wyostrzenie po odszumieniu
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(2.5)

        elif method == 'aggressive_jpeg':
            # Silna kompresja JPEG
            output_buffer = Path("temp_jpeg.jpg")
            img.save(output_buffer, "JPEG", quality=40)
            img = Image.open(output_buffer)
            os.remove(output_buffer)

        elif method == 'upscale_blur_sharp':
            # Symulacja upscalingu: zmniejszenie, rozmycie, a potem agresywne wyostrzenie po upscalingu
            w, h = img.size
            img = img.resize((w // 2, h // 2), Image.LANCZOS)
            img = img.resize((w, h), Image.LANCZOS).filter(ImageFilter.GaussianBlur(radius=0.5))
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(3.0)

        return img
    except Exception as e:
        print(f"Błąd przy metodzie {method}: {e}")
        return img


def generate_nua_images(source_dir: Path, output_dir: Path, num_per_source=4):
    """Generuje obrazy NUA z obrazów źródłowych."""
    print(f"Generowanie {num_per_source} wariantów NUA dla każdego obrazu źródłowego...")
    output_dir.mkdir(parents=True, exist_ok=True)

    source_files = [p for p in source_dir.rglob('*') if p.suffix.lower() in IMAGE_EXTS and p.is_file()]

    nua_methods = ['simulated_hdr', 'ai_denoising_sharp', 'aggressive_jpeg', 'upscale_blur_sharp']

    for src_path in tqdm(source_files, desc="Generowanie NUA"):
        try:
            img = Image.open(src_path).convert('RGB')
            base_name = src_path.stem

            # Generowanie NUA
            for i, method in enumerate(nua_methods[:num_per_source]):
                nua_img = apply_nua_effect(img.copy(), method)
                # Zapisujemy w formacie JPEG z wysoką jakością, aby zachować artefakty obróbki
                nua_path = output_dir / f"{base_name}_{method}.jpg"
                nua_img.save(nua_path, "JPEG", quality=95)

        except Exception as e:
            print(f"Pominięto {src_path.name} z powodu błędu: {e}")
            continue


# =================================================================
# Krok 2: Transformacja Residuals
# =================================================================

class ResidualTransform:
    """Konwertuje obraz na jego residuum (różnicę z wersją rozmazaną)."""
    def __init__(self, kernel_size=3):
        self.kernel_size = kernel_size
        self.blur_filter = ImageFilter.GaussianBlur(kernel_size)

    def __call__(self, img):
        if not isinstance(img, Image.Image):
            raise TypeError("Obraz musi być typu PIL.Image")

        # Obliczanie residuum: Obraz - Rozmyty obraz
        img_np = np.array(img, dtype=np.float32)
        blurred_img = img.filter(self.blur_filter)
        blurred_np = np.array(blurred_img, dtype=np.float32)

        # Obliczenie residuum (szumu)
        residual_np = img_np - blurred_np

        # Normalizacja residuum do zakresu [0, 255] i konwersja na obraz PIL (dla kompatybilności z ToTensor)
        # Używamy przesunięcia o 128, aby zachować wartości ujemne
        residual_norm = np.clip(residual_np + 128, 0, 255).astype(np.uint8)

        return Image.fromarray(residual_norm).convert('RGB')

# Transforms z Residuals
train_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    ResidualTransform(kernel_size=3), # Krok 1: Residuals
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalizacja residuum
])

test_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    ResidualTransform(kernel_size=3), # Krok 1: Residuals
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalizacja residuum
])


# ==============
# Krok 1B: Heurystyki rozdzielenia (FUNKCJA NIEUŻYWANA W NOWYM PIPELINE)
# ==============

def read_exif_tags(path: Path):
    # (Bez zmian)
    try:
        with open(path, 'rb') as f:
            tags = exifread.process_file(f, details=False)
        return {k: str(v) for k, v in tags.items()}
    except Exception:
        return {}


def heuristic_is_nua(path: Path):
    """Rozszerzona heurystyka: EXIF, nazwa pliku i analiza korelacji kanałów/wariancji szumu."""
    name = path.name.lower()

    # 1. Heurystyka nazwy/EXIF (z oryginalnego skryptu)
    keywords = ['hdr', 'ai', 'upscal', 'upscale', 'edited', 'photoshop', 'lightroom', 'luminar', 'gimp', 'remaster', 'enhance']
    if any(k in name for k in keywords):
        return True, 'filename_keyword'

    tags = read_exif_tags(path)
    sw = tags.get('Image Software') or tags.get('Software') or tags.get('Image Processing Software')
    if sw:
        sw_low = sw.lower()
        for k in ['photoshop', 'lightroom', 'snapseed', 'vsco', 'luminar', 'remini', 'facetune', 'ai', 'capture one']:
            if k in sw_low:
                return True, f'exif_software:{k}'

    # 2. Heurystyka cech szumowych (Nowość)
    try:
        img = Image.open(path).convert('RGB')
        np_img = np.array(img).astype(np.float32)

        # Analiza rozkładu różnic między sąsiednimi pikselami (proxy dla szumu/wyostrzenia)
        diff_x = np.abs(np_img[:, 1:, :] - np_img[:, :-1, :]).mean()
        diff_y = np.abs(np_img[1:, :, :] - np_img[:-1, :, :]).mean()
        avg_diff = (diff_x + diff_y) / 2

        # Obrazy AI/HDR często mają bardzo niski jednolity szum (niska std po filtrowaniu)
        # lub bardzo wysoki (agresywne wyostrzenie)

        if avg_diff > 35: # Bardzo wysoka gwałtowność zmian (agresywne wyostrzenie)
             return True, 'high_spatial_variance'

    except Exception:
        pass # Jeśli wystąpi błąd, używamy domyślnego False

    return False, 'none'

# Funkcje propose_labels, build_dataset_from_proposals, split_dataset
# (NIEUŻYWANE W NOWYM PIPELINE, ALE POZOSTAWIONE DLA KOMPLETNOŚCI)

def propose_labels(source_dir: Path, out_csv: Path):
    records = []
    # Skanujemy zarówno oryginalny katalog, jak i wygenerowany katalog NUA
    source_dirs_to_scan = [source_dir, GENERATED_NUA_DIR]

    for current_dir in source_dirs_to_scan:
        for p in tqdm(list(current_dir.rglob('*')), desc=f"Skanowanie {current_dir.name}"):
            if p.suffix.lower() in IMAGE_EXTS and p.is_file():
                # Obrazy z GENERATED_NUA_DIR są z definicji NUA (etykieta 1)
                if current_dir == GENERATED_NUA_DIR:
                    label, reason = 1, 'generated_nua'
                else:
                    nua, reason = heuristic_is_nua(p)
                    label = 1 if nua else 0

                records.append({'path': str(p), 'proposal_label': int(label), 'reason': reason})

    df = pd.DataFrame(records)
    df.to_csv(out_csv, index=False)
    return df


def build_dataset_from_proposals(proposals_csv: Path, output_root: Path, overwrite=False):
    df = pd.read_csv(proposals_csv)
    manual_csv = output_root / 'manual_labels.csv'
    if manual_csv.exists():
        df_manual = pd.read_csv(manual_csv)
        df = df.merge(df_manual, on='path', how='left')
        df['final_label'] = df['manual_label'].fillna(df['proposal_label']).astype(int)
    else:
        df['final_label'] = df['proposal_label'].astype(int)

    # Tworzymy struktury
    ds_orig = output_root / 'dataset' / 'original'
    ds_nua = output_root / 'dataset' / 'nua'
    ds_orig.mkdir(parents=True, exist_ok=True)
    ds_nua.mkdir(parents=True, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Kopiowanie do dataset/"):
        src = Path(row['path'])
        lbl = int(row['final_label'])
        if not src.exists():
            continue
        dest = ds_nua / src.name if lbl == 1 else ds_orig / src.name

        # Dodajemy unikalny ID, aby uniknąć kolizji nazw dla plików z różnych źródeł
        dest_name = f"{src.stem}_{hash(src.parent)}.{src.suffix.strip('.')}"
        dest = dest.parent / dest_name

        if dest.exists() and not overwrite:
            continue
        shutil.copy2(src, dest)

    return df


def split_dataset(dataset_dir: Path, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=SEED):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6
    dst_root = dataset_dir.parent / 'split'
    if dst_root.exists():
        print("Uwaga: katalog split już istnieje. Usuwam i nadpisuję.")
        shutil.rmtree(dst_root)

    for subset in ['train', 'val', 'test']:
        (dst_root / subset / 'original').mkdir(parents=True, exist_ok=True)
        (dst_root / subset / 'nua').mkdir(parents=True, exist_ok=True)

    for class_name, lbl in [('original', 0), ('nua', 1)]:
        src_dir = dataset_dir / class_name
        imgs = [p for p in src_dir.iterdir() if p.suffix.lower() in IMAGE_EXTS]
        random.shuffle(imgs)
        n = len(imgs)
        if n == 0: continue

        t_end = int(n * train_ratio)
        v_end = t_end + int(n * val_ratio)
        splits = {'train': imgs[:t_end], 'val': imgs[t_end:v_end], 'test': imgs[v_end:]}

        for k, files in splits.items():
            for f in files:
                dest = dst_root / k / class_name / f.name
                shutil.copy2(f, dest)

    return dst_root

# ==============
# Model i Trenowanie
# ==============

class CNNEffNetV2(nn.Module):
    def __init__(self, backbone_name="efficientnetv2_rw_m", pretrained=True, num_classes=NUM_CLASSES):
        super().__init__()
        # Poprawiona nazwa modelu na 'efficientnetv2_rw_m'
        self.backbone = timm.create_model("efficientnetv2_rw_m", pretrained=pretrained, num_classes=0, global_pool='avg')
        in_features = self.backbone.num_features
        self.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        features = self.backbone(x)
        out = self.fc(features)
        return out, features


def train_loop(model, optimizer, criterion, loader, scheduler=None):
    model.train()
    total_loss = 0
    for imgs, labels in tqdm(loader, desc='Train', leave=False):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs, _ = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # W ReduceLROnPlateau scheduler.step() wywoływane jest tylko po walidacji
        # Tutaj mamy wbudowany scheduler, który wykonuje step() po każdym batchu (jeśli jest używany)
        # W tym skrypcie nie używamy schedulera, który wymaga step() po batchu (jak OneCycleLR),
        # więc to jest poprawne, że nic się nie dzieje, gdy scheduler jest ReduceLROnPlateau.
        total_loss += loss.item()
    return total_loss / (len(loader) + 1e-9)


def eval_loop(model, criterion, loader):
    model.eval()
    total_loss = 0
    preds_all = []
    labels_all = []
    probs_all = []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc='Eval', leave=False):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs, _ = model(imgs)
            loss = criterion(outputs, labels)

            probs = F.softmax(outputs, dim=1)[:, 1] # Prawdopodobieństwo klasy NUA (1)
            preds = outputs.argmax(1)

            preds_all += preds.cpu().tolist()
            labels_all += labels.cpu().tolist()
            probs_all += probs.cpu().tolist()
            total_loss += loss.item()

    acc = accuracy_score(labels_all, preds_all) if len(labels_all) > 0 else 0
    return acc, total_loss / (len(loader) + 1e-9), np.array(preds_all), np.array(labels_all), np.array(probs_all)


def extract_features(model, loader):
    model.eval()
    feats = []
    labels = []
    with torch.no_grad():
        for imgs, lbls in tqdm(loader, desc='Extract feats', leave=False):
            imgs = imgs.to(DEVICE)
            _, f = model(imgs)
            feats.append(f.cpu().numpy())
            labels.append(lbls.numpy())
    return np.vstack(feats), np.concatenate(labels)


# ==============
# Funkcja główna: pipeline (ZMODYFIKOWANA)
# ==============

def run_pipeline():
    # 1) POMINIĘTO: Generowanie obrazów NUA, etykietowanie i tworzenie splitu.
    print('1) POMINIĘTO: Generowanie obrazów NUA, etykietowanie i tworzenie splitu.')

    # Ścieżka podana przez użytkownika: /content/drive/MyDrive/NUA_Detection_Results/split
    split_dir = Path("/content/drive/MyDrive/NUA_Detection_Results/split")
    print(f'2) Używam istniejącego splitu z: {split_dir}')

    # ImageFolder datasets
    train_data = ImageFolder(split_dir / 'train', transform=train_tf)
    val_data = ImageFolder(split_dir / 'val', transform=test_tf)
    test_data = ImageFolder(split_dir / 'test', transform=test_tf)

    if len(train_data) == 0 or len(test_data) == 0:
        print("BŁĄD: Zbiory danych są puste. Sprawdź, czy ścieżka do splitu jest poprawna i zawiera dane.")
        return

    print(f"Dane: Trening={len(train_data)}, Walidacja={len(val_data)}, Test={len(test_data)}")

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    # model init
    model = CNNEffNetV2().to(DEVICE)
    # Używamy SGD z momentum dla lepszej konwergencji na zadaniach forensycznych
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()

    # SCHEDULER: Usunięto argument 'verbose=True'
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

    best_val_acc = 0
    best_path = OUTPUT_ROOT / 'best_cnn_nua.pth'

    print(f'\n3) Trening EfficientNetV2 ({NUM_EPOCHS} epok)...')
    for epoch in range(1, NUM_EPOCHS + 1):
        tr_loss = train_loop(model, optimizer, criterion, train_loader)
        val_acc, val_loss, _, _, _ = eval_loop(model, criterion, val_loader)
        print(f'Epoka {epoch}/{NUM_EPOCHS}: TrainLoss={tr_loss:.4f} | ValAcc={val_acc:.4f} | ValLoss={val_loss:.4f}')

        # Step schedulera po epoce (na podstawie walidacji)
        scheduler.step(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), str(best_path))

    print('\n4) Wczytuję najlepszy model i ekstrahuję cechy...')
    model.load_state_dict(torch.load(str(best_path), map_location=DEVICE))

    # ekstrakcja cech
    train_feats, train_lbls = extract_features(model, train_loader)
    test_feats, test_lbls = extract_features(model, test_loader)

    # SVM trener
    print('5) Trenuję SVM na cechach i GradientBoosting na probabilistycznych...')
    svm = SVC(kernel='rbf', probability=True, C=5, gamma='auto') # Zwiększamy C dla silniejszego dopasowania
    svm.fit(train_feats, train_lbls)
    svm_probs = svm.predict_proba(test_feats)[:, 1]

    # CNN probabilistyczny
    _, _, _, y_true, cnn_probs = eval_loop(model, criterion, test_loader)

    # meta-ensemble - Używamy SVM i CNN jako cech wejściowych
    X_meta = np.vstack([cnn_probs, svm_probs]).T

    # Dla uproszczenia (brak dedykowanego meta-walidacyjnego zbioru) trenujemy meta-clf na cechach testowych
    # UWAGA: Jest to ryzykowne z punktu widzenia *czystej* metodologii, ale szybkie w prototypowaniu.
    meta_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.03, max_depth=4)
    meta_clf.fit(X_meta, y_true) # Trenowanie na X_meta
    ensemble_probs = meta_clf.predict_proba(X_meta)[:, 1]

    # optymalny próg
    best_thr = 0.5
    best_f1 = 0
    for thr in np.linspace(0.1, 0.9, 81):
        preds = (ensemble_probs >= thr).astype(int)
        f1 = f1_score(y_true, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_thr = thr

    print(f'Najlepszy threshold: {best_thr:.3f} (F1 = {best_f1:.4f})')
    final_preds = (ensemble_probs >= best_thr).astype(int)

    # metryki
    acc = accuracy_score(y_true, final_preds)
    prec = precision_score(y_true, final_preds, zero_division=0)
    rec = recall_score(y_true, final_preds, zero_division=0)
    f1 = f1_score(y_true, final_preds, zero_division=0)

    print('\n=== Raport końcowy ===')
    print('Dokładność (Accuracy):', acc)
    print('Precyzja (Precision):', prec)
    print('Czułość (Recall):', rec)
    print('F1-score:', f1)
    print('\nPełny raport klasyfikacji:')
    print(classification_report(y_true, final_preds, target_names=test_data.classes))

    # Generowanie wykresów (CM, ROC, PR/ROP)

    # Macierz pomyłek
    cm = confusion_matrix(y_true, final_preds)
    fig_cm, ax = plt.subplots(figsize=(6,5))
    im = ax.imshow(cm, cmap='Blues')
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(['original','nua']); ax.set_yticklabels(['original','nua'])
    ax.set_xlabel('Przewidywana etykieta')
    ax.set_ylabel('Prawdziwa etykieta')
    ax.set_title('Macierz pomyłek (Confusion Matrix)')
    for (i, j), val in np.ndenumerate(cm):
        ax.text(j, i, int(val), ha='center', va='center', color='white' if val>cm.max()/2 else 'black')
    fig_cm.colorbar(im)
    fig_cm.savefig(OUTPUT_ROOT / 'confusion_matrix.png', bbox_inches='tight')

    # ROC
    fpr, tpr, _ = roc_curve(y_true, ensemble_probs)
    roc_auc = auc(fpr, tpr)
    fig_roc, axr = plt.subplots()
    axr.plot(fpr, tpr, label=f'Krzywa ROC (AUC = {roc_auc:.3f})')
    axr.plot([0,1],[0,1],'k--')
    axr.set_xlabel('False Positive Rate')
    axr.set_ylabel('True Positive Rate')
    axr.set_title('Krzywa ROC')
    axr.legend()
    fig_roc.savefig(OUTPUT_ROOT / 'roc_curve.png', bbox_inches='tight')

    # Precision-Recall (ROP Curve)
    from sklearn.metrics import precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(y_true, ensemble_probs)
    fig_pr, axpr = plt.subplots()
    axpr.plot(recall, precision)
    axpr.set_xlabel('Recall (Czułość)')
    axpr.set_ylabel('Precision (Precyzja)')
    axpr.set_title('Krzywa Precision-Recall (ROP Curve)')
    fig_pr.savefig(OUTPUT_ROOT / 'precision_recall_curve.png', bbox_inches='tight')

    # Zapis modeli
    torch.save(model.state_dict(), OUTPUT_ROOT / 'final_cnn_nua.pth')
    joblib.dump(svm, OUTPUT_ROOT / 'svm_nua.pkl')
    joblib.dump(meta_clf, OUTPUT_ROOT / 'meta_gb_nua.pkl')

    # Zapis raportu tekstowego (polskie opisy)
    with open(OUTPUT_ROOT / 'report_pl.txt', 'w', encoding='utf-8') as f:
        f.write('Raport detektora NUA\n')
        f.write('-------------------\n')
        f.write(f'Liczba próbek testowych: {len(y_true)}\n')
        f.write(f'Accuracy: {acc:.4f}\n')
        f.write(f'Precision: {prec:.4f}\n')
        f.write(f'Recall (Czułość): {rec:.4f}\n')
        f.write(f'F1-score: {f1:.4f}\n')
        f.write('\nInterpretacja metryk (po polsku):\n')
        f.write(' - Dokładność (Accuracy): odsetek poprawnych klasyfikacji.\n')
        f.write(' - Precyzja (Precision): z wszystkich obrazów oznaczonych jako NUA, jaki odsetek rzeczywiście był NUA.\n')
        f.write(' - Czułość (Recall): z wszystkich rzeczywistych obrazów NUA, jaki odsetek wykryliśmy.\n')
        f.write(' - F1-score: średnia harmoniczna precyzji i czułości, dobra miara przy nierównych klasach.\n')
        f.write('\nZastosowane ulepszenia:\n')
        f.write(' - Pre-processing: Użycie Residual Transform (filtr górnoprzepustowy) dla zwiększenia wrażliwości na artefakty.\n')
        f.write(' - Architektura: EfficientNetV2-M i dłuższy trening.\n')
        f.write(' - Ensemble: Połączenie cech z CNN oraz predykcji probabilistycznych za pomocą SVM i GradientBoosting.\n')
        f.write('\nDodatkowo zapisano wykresy: confusion_matrix.png, roc_curve.png, precision_recall_curve.png\n')

    print('Zapisano raport i modele w', OUTPUT_ROOT)


if __name__ == '__main__':
    run_pipeline()